In [ ]:
import cv2
import imutils

# Initializing the HOG person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

cap = cv2.VideoCapture('vid.mp4')

# Define minimum width and height
MIN_WIDTH = 600
MIN_HEIGHT = 600

while cap.isOpened():
    # Reading the video stream
    ret, image = cap.read()
    if ret:
        height, width = image.shape[:2]

        # Calculate aspect ratio
        aspect_ratio = width / height

        # Resize the image while maintaining the aspect ratio
        if width < MIN_WIDTH or height < MIN_HEIGHT:
            if width < MIN_WIDTH:
                new_width = MIN_WIDTH
                new_height = int(new_width / aspect_ratio)
            if height < MIN_HEIGHT:
                new_height = MIN_HEIGHT
                new_width = int(new_height * aspect_ratio)
            image = cv2.resize(image, (new_width, new_height))
        else:
            # Resize the image to the maximum width of 400 while maintaining aspect ratio
            image = imutils.resize(image, width=min(400, width))

        # Print the resized frame size for debugging
        print(f"Resized frame size: {image.shape[1]}x{image.shape[0]}")

        # Detecting all the regions in the image that have pedestrians inside it
        (regions, _) = hog.detectMultiScale(image,
                                            winStride=(4, 4),  # Reduced winStride
                                            padding=(8, 8),    # Reduced padding
                                            scale=1.1)         # Increased scale factor

        # Drawing the regions in the image
        for (x, y, w, h) in regions:
            # Filtering out detections with low confidence scores
            if _[regions.tolist().index([x, y, w, h])] > 0.5:
                cv2.rectangle(image, (x, y),
                              (x + w, y + h),
                              (0, 0, 255), 2)

        # Showing the output image
        cv2.imshow("Image", image)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()